In [1]:
import torch

In [2]:
inputs=torch.tensor(
    [
     [0.43,0.15,0.89],#your
     [0.55,0.87,0.66],#journey
     [0.57,0.85,0.64],#starts
     [0.22,0.58,0.33],#with
     [0.77,0.25,0.10],#one
     [0.05,0.80,0.55],#step
    ]
)

In [3]:
w_q=torch.nn.Linear(inputs.shape[1],2,bias=False)
print(w_q)
w_k=torch.nn.Linear(inputs.shape[1],2,bias=False)
w_v=torch.nn.Linear(inputs.shape[1],2,bias=False)

Linear(in_features=3, out_features=2, bias=False)


In [7]:
queries=w_q(inputs)
keys=w_k(inputs)
values=w_v(inputs)
print(queries)
print(queries.T)

tensor([[ 0.0903, -0.1535],
        [ 0.2760, -0.3187],
        [ 0.2615, -0.3105],
        [ 0.2004, -0.1990],
        [-0.0732, -0.0765],
        [ 0.3485, -0.2882]], grad_fn=<MmBackward0>)
tensor([[ 0.0903,  0.2760,  0.2615,  0.2004, -0.0732,  0.3485],
        [-0.1535, -0.3187, -0.3105, -0.1990, -0.0765, -0.2882]],
       grad_fn=<PermuteBackward0>)


In [5]:
attn_score=queries@keys.T
print(attn_score)

tensor([[ 0.0232, -0.0178, -0.0144, -0.0245,  0.0502, -0.0569],
        [ 0.0096, -0.0810, -0.0726, -0.0763,  0.0983, -0.1585],
        [ 0.0126, -0.0752, -0.0672, -0.0722,  0.0963, -0.1511],
        [-0.0063, -0.0646, -0.0589, -0.0557,  0.0595, -0.1118],
        [ 0.0631,  0.0501,  0.0499,  0.0219,  0.0331,  0.0255],
        [-0.0344, -0.1225, -0.1134, -0.0974,  0.0822, -0.1884]],
       grad_fn=<MmBackward0>)


In [9]:
attn_weight=torch.softmax(attn_score/keys.shape[1]**0.5,dim=1)
print(attn_weight)

tensor([[0.1702, 0.1653, 0.1657, 0.1645, 0.1735, 0.1608],
        [0.1731, 0.1624, 0.1634, 0.1630, 0.1844, 0.1537],
        [0.1731, 0.1626, 0.1636, 0.1630, 0.1836, 0.1541],
        [0.1705, 0.1636, 0.1643, 0.1647, 0.1786, 0.1583],
        [0.1693, 0.1678, 0.1678, 0.1645, 0.1658, 0.1649],
        [0.1717, 0.1613, 0.1624, 0.1642, 0.1864, 0.1540]],
       grad_fn=<SoftmaxBackward0>)


In [ ]:
#use pytorch tril function to diagonalize
context_length=inputs.shape[0]
mask1=torch.tril(torch.ones(context_length,context_length))
print(mask1)

tensor([[1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0.]])


In [8]:
#use element wise multiplication not matrix
masked_attn_score=attn_weight*mask1
print(masked_attn_score)

#but this attention score is not fully masked or optimal becaus the softmax normalization adds influence on its from the previous attn score so use upper negative infinity mask instead of zero

tensor([[0.1751, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1784, 0.1528, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1782, 0.1531, 0.1539, 0.0000, 0.0000, 0.0000],
        [0.1733, 0.1579, 0.1584, 0.1699, 0.0000, 0.0000],
        [0.1708, 0.1637, 0.1639, 0.1669, 0.1702, 0.0000],
        [0.1759, 0.1539, 0.1546, 0.1716, 0.1816, 0.1624]],
       grad_fn=<MulBackward0>)


In [9]:
#normalize the masked attention score  to sum up 1 in each row to get masked attn weight
row_sum=masked_attn_score.sum(dim=1,keepdim=True)
print(row_sum)
masked_attn_weight=masked_attn_score/row_sum
print(masked_attn_weight)

tensor([[0.1751],
        [0.3312],
        [0.4853],
        [0.6596],
        [0.8354],
        [1.0000]], grad_fn=<SumBackward1>)
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5387, 0.4613, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3672, 0.3156, 0.3172, 0.0000, 0.0000, 0.0000],
        [0.2628, 0.2394, 0.2402, 0.2576, 0.0000, 0.0000],
        [0.2045, 0.1959, 0.1961, 0.1997, 0.2038, 0.0000],
        [0.1759, 0.1539, 0.1546, 0.1716, 0.1816, 0.1624]],
       grad_fn=<DivBackward0>)


In [10]:
masked_context_vectore=masked_attn_weight@values
print(masked_context_vectore)

tensor([[ 0.2130, -0.6795],
        [ 0.3920, -0.8111],
        [ 0.4560, -0.8569],
        [ 0.4368, -0.7743],
        [ 0.4082, -0.7247],
        [ 0.4111, -0.7052]], grad_fn=<MmBackward0>)


# Negative infinity masking to avoid any influence of future attention

In [11]:
print(attn_score)

tensor([[-0.0588, -0.1890, -0.1850, -0.1207, -0.0614, -0.1651],
        [-0.1896, -0.4089, -0.4014, -0.2481, -0.1534, -0.3344],
        [-0.1833, -0.3974, -0.3901, -0.2414, -0.1488, -0.3254],
        [-0.1276, -0.2595, -0.2549, -0.1559, -0.0997, -0.2095],
        [-0.0189, -0.0796, -0.0778, -0.0520, -0.0239, -0.0717],
        [-0.1940, -0.3834, -0.3766, -0.2293, -0.1491, -0.3075]],
       grad_fn=<MmBackward0>)


In [14]:
mask=torch.triu(torch.ones(context_length,context_length),diagonal=1)
print(mask)
masked=attn_score.masked_fill(mask.bool(),-torch.inf)
print(masked)

tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])
tensor([[ 0.0232,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 0.0096, -0.0810,    -inf,    -inf,    -inf,    -inf],
        [ 0.0126, -0.0752, -0.0672,    -inf,    -inf,    -inf],
        [-0.0063, -0.0646, -0.0589, -0.0557,    -inf,    -inf],
        [ 0.0631,  0.0501,  0.0499,  0.0219,  0.0331,    -inf],
        [-0.0344, -0.1225, -0.1134, -0.0974,  0.0822, -0.1884]],
       grad_fn=<MaskedFillBackward0>)


In [13]:
masked_attn_weight=torch.softmax(masked/keys.shape[1],dim=1)
print(masked_attn_weight)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5274, 0.4726, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3571, 0.3209, 0.3220, 0.0000, 0.0000, 0.0000],
        [0.2590, 0.2425, 0.2431, 0.2554, 0.0000, 0.0000],
        [0.2032, 0.1971, 0.1973, 0.1998, 0.2027, 0.0000],
        [0.1732, 0.1576, 0.1581, 0.1702, 0.1772, 0.1637]],
       grad_fn=<SoftmaxBackward0>)


# Masking additional attention weight with Dropout

In [14]:
torch.manual_seed(123)
dropout=torch.nn.Dropout(0.5)
print(dropout)
ones=torch.ones(6,6)
print(ones)
print(dropout(ones))

Dropout(p=0.5, inplace=False)
tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])
tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [15]:
attn_dropout=dropout(attn_weight)
print(attn_dropout)

tensor([[0.3503, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.3072, 0.0000, 0.3661, 0.3221],
        [0.0000, 0.3063, 0.3079, 0.3420, 0.3652, 0.3223],
        [0.0000, 0.0000, 0.3169, 0.3398, 0.0000, 0.3272],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.3404, 0.3291],
        [0.3519, 0.3078, 0.0000, 0.3432, 0.3632, 0.3247]],
       grad_fn=<MulBackward0>)


# Causal attention Class with Batches

In [18]:
batch=torch.stack((inputs,inputs),dim=0)
print(batch)
print(batch.shape)

tensor([[[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]],

        [[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]]])
torch.Size([2, 6, 3])


In [17]:
print(batch.transpose(1,2))#change row(1) with colmn(2)

tensor([[[0.4300, 0.5500, 0.5700, 0.2200, 0.7700, 0.0500],
         [0.1500, 0.8700, 0.8500, 0.5800, 0.2500, 0.8000],
         [0.8900, 0.6600, 0.6400, 0.3300, 0.1000, 0.5500]],

        [[0.4300, 0.5500, 0.5700, 0.2200, 0.7700, 0.0500],
         [0.1500, 0.8700, 0.8500, 0.5800, 0.2500, 0.8000],
         [0.8900, 0.6600, 0.6400, 0.3300, 0.1000, 0.5500]]])


In [20]:
class Causal_Attention(torch.nn.Module):
    def __init__(self,in_d,out_d,context_length,dropout_percent):
        super().__init__()
        self.w_q=torch.nn.Linear(in_d,out_d)
        self.w_k=torch.nn.Linear(in_d,out_d)
        self.w_v=torch.nn.Linear(in_d,out_d)
        self.dropout=torch.nn.Dropout(dropout_percent)
        self.register_buffer('mask',torch.triu(torch.ones(context_length,context_length),diagonal=1))

    
    def forward(self,input_embedding):
        
        b,num_token,d_in=input_embedding.shape
        queries=self.w_q(input_embedding)
        keys=self.w_k(input_embedding)
        values=self.w_v(input_embedding)

        print(keys.shape[-1])

        attn_score=queries@keys.transpose(1,2)

        attn_score.masked_fill_(self.mask.bool()[:context_length, :context_length],-torch.inf)
        
        print('attn_score',attn_score)
        attn_weight=torch.softmax(attn_score/keys.shape[-1]**0.5,dim=-1)
        print('attn_weight',attn_weight)
        attn_weight=self.dropout(attn_weight)

        masked_context_vectore=attn_weight@values
        return masked_context_vectore

        
        

        
        


In [24]:
in_d,out_d=3,3
print(in_d)

3


In [27]:
torch.manual_seed(123)
context_length=batch.shape[1]
print(context_length)
ca=Causal_Attention(in_d,6,context_length,0.0)
print(ca)

ca.forward(batch)


6
Causal_Attention(
  (w_q): Linear(in_features=3, out_features=6, bias=True)
  (w_k): Linear(in_features=3, out_features=6, bias=True)
  (w_v): Linear(in_features=3, out_features=6, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
)
6
attn_score tensor([[[0.3992,   -inf,   -inf,   -inf,   -inf,   -inf],
         [0.3519, 0.3963,   -inf,   -inf,   -inf,   -inf],
         [0.3568, 0.4077, 0.4128,   -inf,   -inf,   -inf],
         [0.1898, 0.2388, 0.2436, 0.1718,   -inf,   -inf],
         [0.3699, 0.5750, 0.5811, 0.4127, 0.5952,   -inf],
         [0.1539, 0.1378, 0.1424, 0.1046, 0.2145, 0.0622]],

        [[0.3992,   -inf,   -inf,   -inf,   -inf,   -inf],
         [0.3519, 0.3963,   -inf,   -inf,   -inf,   -inf],
         [0.3568, 0.4077, 0.4128,   -inf,   -inf,   -inf],
         [0.1898, 0.2388, 0.2436, 0.1718,   -inf,   -inf],
         [0.3699, 0.5750, 0.5811, 0.4127, 0.5952,   -inf],
         [0.1539, 0.1378, 0.1424, 0.1046, 0.2145, 0.0622]]],
       grad_fn=<MaskedFillBackward0>

tensor([[[ 0.3219,  0.2094, -0.9932,  0.0712,  0.1319,  0.1719],
         [ 0.4575,  0.2406, -1.1512,  0.1678,  0.3740,  0.0621],
         [ 0.5021,  0.2493, -1.1980,  0.2061,  0.4552,  0.0246],
         [ 0.4403,  0.2167, -1.1513,  0.2155,  0.4965,  0.0534],
         [ 0.4287,  0.1912, -1.0795,  0.2991,  0.5302,  0.0399],
         [ 0.4024,  0.1877, -1.0947,  0.2588,  0.5358,  0.0623]],

        [[ 0.3219,  0.2094, -0.9932,  0.0712,  0.1319,  0.1719],
         [ 0.4575,  0.2406, -1.1512,  0.1678,  0.3740,  0.0621],
         [ 0.5021,  0.2493, -1.1980,  0.2061,  0.4552,  0.0246],
         [ 0.4403,  0.2167, -1.1513,  0.2155,  0.4965,  0.0534],
         [ 0.4287,  0.1912, -1.0795,  0.2991,  0.5302,  0.0399],
         [ 0.4024,  0.1877, -1.0947,  0.2588,  0.5358,  0.0623]]],
       grad_fn=<UnsafeViewBackward0>)

# Multi head attention wrapper class

In [28]:
class MultiHeadAttentionWrapper(torch.nn.Module):

    def __init__(self,in_d,out_d,context_length,dropout_percent,num_head):

        super().__init__()
        self.heads=torch.nn.ModuleList([Causal_Attention(in_d,out_d,context_length,dropout_percent) for _ in range(num_head)])
        print('self head',self.heads)
    def forward(self,inputs):
        return torch.cat([head(inputs) for head in self.heads],dim=-1)

In [29]:
torch.manual_seed(123)
context_length=batch.shape[1]
print(context_length)
mha=MultiHeadAttentionWrapper(in_d,6,context_length,0.0,num_head=2)

context_vec=mha(batch)
print('context_veccccccccc',context_vec)




6
self head ModuleList(
  (0-1): 2 x Causal_Attention(
    (w_q): Linear(in_features=3, out_features=6, bias=True)
    (w_k): Linear(in_features=3, out_features=6, bias=True)
    (w_v): Linear(in_features=3, out_features=6, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
)
6
attn_score tensor([[[0.3992,   -inf,   -inf,   -inf,   -inf,   -inf],
         [0.3519, 0.3963,   -inf,   -inf,   -inf,   -inf],
         [0.3568, 0.4077, 0.4128,   -inf,   -inf,   -inf],
         [0.1898, 0.2388, 0.2436, 0.1718,   -inf,   -inf],
         [0.3699, 0.5750, 0.5811, 0.4127, 0.5952,   -inf],
         [0.1539, 0.1378, 0.1424, 0.1046, 0.2145, 0.0622]],

        [[0.3992,   -inf,   -inf,   -inf,   -inf,   -inf],
         [0.3519, 0.3963,   -inf,   -inf,   -inf,   -inf],
         [0.3568, 0.4077, 0.4128,   -inf,   -inf,   -inf],
         [0.1898, 0.2388, 0.2436, 0.1718,   -inf,   -inf],
         [0.3699, 0.5750, 0.5811, 0.4127, 0.5952,   -inf],
         [0.1539, 0.1378, 0.1424, 0.1046, 0.2145, 

# MultiHead attention with weight splits

In [33]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(self,d_in,context_length,d_out,num_heads,dropout_percent):
        super().__init__()
        assert (d_out%num_heads==0)

        self.d_out=d_out
        self.num_heads=num_heads
        self.head_dim=self.d_out//self.num_heads
        self.wk=torch.nn.Linear(d_in,d_out)
        self.wq=torch.nn.Linear(d_in,d_out)
        self.wv=torch.nn.Linear(d_in,d_out)
        self.out_proj=torch.nn.Linear(d_out,d_out)
        self.dropout=torch.nn.Dropout(dropout_percent)
        self.register_buffer('mask',torch.triu(torch.ones(context_length,context_length),diagonal=1))

    def forward(self,batch):
        b,num_token,d_in=batch.shape
        keys=self.wk(batch)# shape: (b,num_token,d_out)
        queries=self.wq(batch)
        values=self.wv(batch)
        print(keys)
        #implicitly split the matrix by adding 'num_heads' dimension
        #(b,num_token,d_out)--->(b,num_token,num_head,head_dim)

        keys=keys.view(b,num_token,self.num_heads,self.head_dim)
        queries=queries.view(b,num_token,self.num_heads,self.head_dim)
        values=values.view(b,num_token,self.num_heads,self.head_dim)
        print(keys)
        #transpose: (b,num_token,num_head,head_dim)---->(b,num_head,num_token,head_dim)

        keys=keys.transpose(1,2)
        print(keys)
        queries=queries.transpose(1,2)
        values=values.transpose(1,2)

      
        
        #compute self attention with causal mask
        attn_score=queries@keys.transpose(2,3) #dot product for each head
        mask_bool=self.mask.bool()[:num_token,:num_token]
   
        attn_score.masked_fill_(mask_bool,-torch.inf)
  
        attn_weight=torch.softmax(attn_score/keys.shape[-1]**0.5,dim=-1)
        attn_weight=self.dropout(attn_weight)

        #shape (b,num_head,num_token,head_dim)---->(b,num_token,num_head,head_dim)
        context_vec=(attn_weight@values).transpose(1,2)

        #combine heads, where self.d_out=self.head_num*self.head_dim
        context_vec=context_vec.contiguous().view(b,num_token,self.d_out)
        context_vec=self.out_proj(context_vec)#optional
        

        print(context_vec)

       

        


       

        

In [34]:
b,context_length,d_in=batch.shape
MHA=MultiHeadAttention(d_in,context_length,d_out=6,num_heads=2,dropout_percent=0.5)

print(batch.shape)
MHA(batch)

torch.Size([2, 6, 3])
tensor([[[ 0.1627, -0.4244,  0.3030,  1.1157,  0.1149,  0.1361],
         [-0.2507, -0.4525,  0.6040,  1.3664,  0.2574, -0.0498],
         [-0.2412, -0.4527,  0.6213,  1.3642,  0.2674, -0.0268],
         [-0.2099, -0.4540,  0.5598,  0.9916,  0.2676,  0.0684],
         [-0.0014, -0.4549,  0.9066,  1.1026,  0.4534,  0.4946],
         [-0.3072, -0.4520,  0.3942,  1.0423,  0.1694, -0.1663]],

        [[ 0.1627, -0.4244,  0.3030,  1.1157,  0.1149,  0.1361],
         [-0.2507, -0.4525,  0.6040,  1.3664,  0.2574, -0.0498],
         [-0.2412, -0.4527,  0.6213,  1.3642,  0.2674, -0.0268],
         [-0.2099, -0.4540,  0.5598,  0.9916,  0.2676,  0.0684],
         [-0.0014, -0.4549,  0.9066,  1.1026,  0.4534,  0.4946],
         [-0.3072, -0.4520,  0.3942,  1.0423,  0.1694, -0.1663]]],
       grad_fn=<ViewBackward0>)
tensor([[[[ 0.1627, -0.4244,  0.3030],
          [ 1.1157,  0.1149,  0.1361]],

         [[-0.2507, -0.4525,  0.6040],
          [ 1.3664,  0.2574, -0.0498]],

  